In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
import torch



In [10]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal

from seiz_eeg.dataset import EEGDataset

import os
import random
from datetime import datetime

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

import argparse

from sklearn.metrics import f1_score
from tqdm import tqdm

from src.utils import load_config, load_eeg_data, load_graph

config = load_config("configs/tgcn.yaml")


dataset_tr, dataset_val, train_df = load_eeg_data(
    config["data_path"],
    config["train_parquet_file"],
    config["val_parquet_file"],
    config["signal_processing"]["filtering_type"],
)

"""# Model"""

# Set up device
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device = "cpu"
print("Using device:", device)

# Load distances of nodes
edge_index, edge_weight = load_graph(
    config["data_path"], config["distances_csv_file"]
)
edge_index = edge_index.to(device)
edge_weight = edge_weight.to(device)


Loading Data
Time taken for filtering: 162.69532322883606
Using device: cpu


In [39]:
dataset_tr[0]

(array([[-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074],
        [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074],
        [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074],
        ...,
        [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074],
        [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074],
        [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
         -18.42068074, -18.42068074]]),
 0)

In [45]:
# Assuming you have an EEGDataset object
def apply_smote_to_eeg_dataset(eeg_dataset):
    # Extract and flatten data
    X = []
    y = []
    original_shape = None

    for i in range(len(eeg_dataset)):
        data, label = eeg_dataset[i]
        if original_shape is None:
            original_shape = data.shape
        X.append(data.flatten())
        y.append(label)

    X = np.array(X)
    y = np.array(y)

    # Apply SMOTE
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Reshape X back to original EEG format
    X_resampled = X_resampled.reshape(-1, *original_shape)

    # Combine X and y back together as list of tuples
    combined_dataset = list(zip(torch.from_numpy(X_resampled), torch.from_numpy(y_resampled)))

    return combined_dataset

hh = apply_smote_to_eeg_dataset(dataset_tr)

In [47]:
loader_tr = DataLoader(
    hh, batch_size=config["training"]["batch_size"], shuffle=True
)


for x, y in loader_tr:
    print(y)
    break

tensor([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1,
        0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 1, 0])


In [35]:
dataset_tr[0][0]

array([[-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074],
       [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074],
       [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074],
       ...,
       [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074],
       [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074],
       [-18.42068074, -18.42068074, -18.42068074, ..., -18.42068074,
        -18.42068074, -18.42068074]])

In [2]:
dataset_tr = [
    Data(x=x, y=y, edge_index=edge_index, edge_weight=edge_weight)
    for x, y in dataset_tr
]
dataset_val = [Data(x=x, y=y, edge_index=edge_index, edge_weight=edge_weight) for x, y in dataset_val]


loader_tr = DataLoader(
    dataset_tr, batch_size=config["training"]["batch_size"], shuffle=True
)
loader_val = DataLoader(
    dataset_val, batch_size=config["training"]["batch_size"], shuffle=False
    )

/opt/homebrew/Caskroom/miniforge/base/envs/gnn_epfl/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [3]:
for batch in loader_tr:
    print(torch.tensor(batch.x).shape)
    print(torch.tensor(batch.edge_index).shape)
    break

torch.Size([128, 354, 19])
torch.Size([2, 46208])


/var/folders/dy/b1b3v71j30715q_7sr29yn6h0000gn/T/ipykernel_46935/941056253.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  print(torch.tensor(batch.x).shape)
/var/folders/dy/b1b3v71j30715q_7sr29yn6h0000gn/T/ipykernel_46935/941056253.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.tensor(batch.edge_index).shape)


In [7]:
from torch_geometric_temporal.nn.recurrent import TGCN

tgcn = TGCN(19, 200).to(device)

for batch in loader_tr:
    print(torch.tensor(batch.x).shape)
    print(torch.tensor(batch.edge_index).shape)
    tgcn(torch.tensor(batch.x).float().to(device), torch.tensor(batch.edge_index).to(device))
    break

torch.Size([128, 354, 19])
torch.Size([2, 46208])


/var/folders/dy/b1b3v71j30715q_7sr29yn6h0000gn/T/ipykernel_46935/4128504691.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(torch.tensor(batch.edge_index).shape)
/var/folders/dy/b1b3v71j30715q_7sr29yn6h0000gn/T/ipykernel_46935/4128504691.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgcn(torch.tensor(batch.x).float().to(device), torch.tensor(batch.edge_index).to(device))


RuntimeError: index 355 is out of bounds for dimension 0 with size 354

In [2]:
from torch_geometric.nn.models import tgn

In [ ]:
import torch
import numpy as np

# Example distance matrix
D = np.array([
    [0.0, 0.5, 2.1],
    [0.5, 0.0, 1.7],
    [2.1, 1.7, 0.0]
])

# Threshold to determine edges
epsilon = 1.0

# Build edge list based on the threshold
src, dst, weights = [], [], []

for i in range(D.shape[0]):
    for j in range(D.shape[1]):
        if i != j and D[i][j] <= epsilon:
            src.append(i)
            dst.append(j)
            weights.append(1.0 / (1e-5 + D[i][j]))  # optional: inverse distance as weight

# so to represent the adjacency matrix you use edge list and the other the weights?
edge_index = torch.tensor([src, dst], dtype=torch.long)
edge_weight = torch.tensor(weights, dtype=torch.float32)


In [4]:
x = torch.randn((D.shape[0], 16))  # 3 nodes, 16 features each


In [5]:
from torch_geometric.nn import GCNConv

conv = GCNConv(in_channels=16, out_channels=32)

# Forward pass
x_out = conv(x, edge_index, edge_weight)


In [6]:
print(x_out)

tensor([[-3.2089e-01, -6.3288e-01, -1.8896e-01, -2.3628e-01, -2.8817e-01,
          3.4328e-01,  6.7342e-01, -2.4728e-01, -3.5239e-01, -2.4342e-01,
          3.2315e-01,  5.0067e-01, -2.5791e-01,  6.0807e-01, -4.8137e-01,
          4.2984e-02, -3.7144e-01, -4.0818e-02,  4.5687e-01,  1.8449e-02,
         -1.5940e-01, -8.6577e-02,  7.2570e-01,  9.0183e-01, -3.8834e-01,
          9.1921e-02, -2.5331e-01,  3.8133e-01, -2.4998e-01, -1.9012e-01,
          3.1951e-02, -4.5143e-01],
        [-3.8717e-01, -5.7393e-01, -1.9860e-01,  2.0425e-01, -2.2687e-01,
          3.6155e-01,  7.2146e-01, -3.6963e-01, -5.7922e-01, -1.8449e-01,
          2.6313e-01,  1.2110e-01, -4.8737e-01,  8.1585e-02, -4.0970e-01,
          1.6158e-01, -1.6580e-01,  3.4525e-01,  2.5812e-01, -9.0533e-02,
         -5.5841e-02, -2.6969e-02,  6.7868e-01,  8.4918e-01, -3.6789e-02,
          8.3061e-01, -1.2569e-01,  2.9951e-02, -5.1312e-01, -1.9402e-01,
          6.2289e-02, -4.6807e-01],
        [-1.9891e+00,  6.9298e-01, -6.33